# Naive Machine Translation and Local Sensitive Hashing

In [58]:
import pdb
import pickle
import string

import time

import nltk
import numpy as np
from nltk.corpus import stopwords, twitter_samples

from utils import cosine_similarity, get_dict, process_tweet
from os import getcwd
import w4_unittest

In [102]:
filePath = f"{getcwd()}/tmp2/"
nltk.data.path.append(filePath)

## Load data

### The embeddings as a dict - key = word, value=embedding

In [59]:
en_embeddings_subset = pickle.load(open("./data/en_embeddings.p", "rb"))
fr_embeddings_subset = pickle.load(open("./data/fr_embeddings.p", "rb"))

### Dictionaries

In [60]:
en_fr_train = get_dict("./data/en-fr.train.txt")
en_fr_test = get_dict("./data/en-fr.test.txt")

/Users/mikael/Library/Mobile Documents/com~apple~CloudDocs/Learning/Deeplearning.ai/Specializations/003-nlp/001-nlp-with-classification-and-vector-spaces/assingments/004-week-4-machine-translation/utils.py:56: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  en = my_file.loc[i][0]
/Users/mikael/Library/Mobile Documents/com~apple~CloudDocs/Learning/Deeplearning.ai/Specializations/003-nlp/001-nlp-with-classification-and-vector-spaces/assingments/004-week-4-machine-translation/utils.py:57: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fr = my_file.loc[i][1]
/Users/mikael/Library/Mobile Documents/com~apple~CloudDocs/Learnin

In [61]:
en_fr_train

{'the': 'la',
 'and': 'et',
 'was': 'était',
 'for': 'pour',
 'that': 'cela',
 'with': 'avec',
 'from': 'depuis',
 'this': 'ce',
 'utc': 'tuc',
 'his': 'son',
 'not': 'pas',
 'are': 'sont',
 'talk': 'parlez',
 'which': 'lequel',
 'also': 'egalement',
 'were': 'étaient',
 'but': 'mais',
 'have': 'ont',
 'one': 'one',
 'new': 'nouveautés',
 'first': 'premiers',
 'page': 'page',
 'you': 'you',
 'they': 'eux',
 'had': 'avais',
 'article': 'article',
 'who': 'who',
 'all': 'all',
 'their': 'leurs',
 'there': 'là',
 'made': 'fabriqué',
 'its': 'son',
 'people': 'personnes',
 'may': 'peut',
 'after': 'aprés',
 'other': 'autres',
 'should': 'devrais',
 'two': 'deux',
 'score': 'partition',
 'her': 'her',
 'can': 'peut',
 'would': 'ferait',
 'more': 'plus',
 'she': 'elle',
 'when': 'quand',
 'time': 'heure',
 'team': 'equipe',
 'american': 'américains',
 'such': 'telles',
 'discussion': 'débat',
 'links': 'liens',
 'only': 'seule',
 'some': 'quelques',
 'see': 'vois',
 'united': 'unies',
 'year

In [62]:
len(en_fr_train)

5000

In [63]:
len(en_fr_test)

1500

In [64]:
def get_matrices(en_fr, french_vecs, english_vecs):
    """
    Input:
        en_fr: English to French dictionary
        french_vecs: French words to their corresponding word embeddings.
        english_vecs: English words to their corresponding word embeddings.
    Output:
        X: a matrix where the columns are the English embeddings.
        Y: a matrix where the columns correspong to the French embeddings.
        R: the projection matrix that minimizes the F norm ||X R -Y||^2.
    """

    X_l = list()
    Y_l = list()

    english_set = set(english_vecs.keys())
    french_set = set(french_vecs.keys())
    french_words = set(en_fr.values())

    for en_word, fr_word in en_fr.items():
        if fr_word in french_set and en_word in english_set:
            en_vec = english_vecs[en_word]
            fr_vec = french_vecs[fr_word]
            X_l.append(en_vec)
            Y_l.append(fr_vec)

    X = np.array(X_l)
    print(X.shape)

    Y = np.array(Y_l)
    print(Y.shape)
    return X, Y

In [65]:
X_train, Y_train = get_matrices(en_fr_train, fr_embeddings_subset, en_embeddings_subset)

(4932, 300)
(4932, 300)


In [66]:
w4_unittest.test_get_matrices(get_matrices)

/Users/mikael/Library/Mobile Documents/com~apple~CloudDocs/Learning/Deeplearning.ai/Specializations/003-nlp/001-nlp-with-classification-and-vector-spaces/assingments/004-week-4-machine-translation/utils.py:56: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  en = my_file.loc[i][0]
/Users/mikael/Library/Mobile Documents/com~apple~CloudDocs/Learning/Deeplearning.ai/Specializations/003-nlp/001-nlp-with-classification-and-vector-spaces/assingments/004-week-4-machine-translation/utils.py:57: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fr = my_file.loc[i][1]


(4932, 300)
(4932, 300)
(1438, 300)
(1438, 300)
 All tests passed


/Users/mikael/Library/Mobile Documents/com~apple~CloudDocs/Learning/Deeplearning.ai/Specializations/003-nlp/001-nlp-with-classification-and-vector-spaces/assingments/004-week-4-machine-translation/utils.py:56: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  en = my_file.loc[i][0]
/Users/mikael/Library/Mobile Documents/com~apple~CloudDocs/Learning/Deeplearning.ai/Specializations/003-nlp/001-nlp-with-classification-and-vector-spaces/assingments/004-week-4-machine-translation/utils.py:57: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fr = my_file.loc[i][1]


## Translations

In [67]:
def compute_loss(X, Y, R):
    """
    Inputs:
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        R: a matrix of dimension (n,n) - transformation matrix from English to French vector space embeddings.
    Outputs:
        L: a matrix of dimension (m,n) - the value of the loss function for given X, Y and R.
    """

    m = X.shape[0]
    diff = X @ R - Y
    diff_squared = np.square(diff)
    sum_diff_squared = np.sum(diff_squared)
    loss = sum_diff_squared / m
    return loss

In [68]:
np.random.seed(123)
m = 10
n = 5
X = np.random.rand(m, n)
Y = np.random.rand(m, n) * 0.1
R = np.random.rand(n, n)
print(
    f"Expected loss for an experiment with random matrices: {compute_loss(X, Y, R):.4f}"
)

Expected loss for an experiment with random matrices: 8.1866


In [69]:
w4_unittest.test_compute_loss(compute_loss)

 All tests passed


## Compute gradiant

In [70]:
def compute_gradient(X, Y, R):
    """
    Inputs:
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        R: a matrix of dimension (n,n) - transformation matrix from English to French vector space embeddings.
    Outputs:
        g: a scalar value - gradient of the loss function L for given X, Y and R.
    """
    m = X.shape[0]

    gradient = X.T @ (X @ R - Y) * (2 / m)
    return gradient

In [71]:
# Testing your implementation.
np.random.seed(123)
m = 10
n = 5
X = np.random.rand(m, n)
Y = np.random.rand(m, n) * 0.1
R = np.random.rand(n, n)
gradient = compute_gradient(X, Y, R)
print(f"First row of the gradient matrix: {gradient[0]}")

First row of the gradient matrix: [1.3498175  1.11264981 0.69626762 0.98468499 1.33828969]


In [72]:
w4_unittest.test_compute_gradient(compute_gradient)

 All tests passed


## Calculate R

In [73]:
def align_embeddings(
    X,
    Y,
    train_steps=100,
    learning_rate=0.0003,
    verbose=True,
    compute_loss=compute_loss,
    compute_gradient=compute_gradient,
):
    """
    Inputs:
        X: a matrix of dimension (m,n) where the columns are the English embeddings. ---> Rows m words, Cols n embeedding features
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        train_steps: positive int - describes how many steps will gradient descent algorithm do.
        learning_rate: positive float - describes how big steps will  gradient descent algorithm do.
    Outputs:
        R: a matrix of dimension (n,n) - the projection matrix that minimizes the F norm ||X R -Y||^2
    """
    np.random.seed(129)

    R = np.random.rand(X.shape[1], X.shape[1])

    for i in range(train_steps):
        if verbose and i % 25 == 0:
            print(f"Loss at iteration {i} i {compute_loss(X, Y, R)}")

        gradient = compute_gradient(X, Y, R)
        R = R - learning_rate * gradient

    return R

In [74]:
np.random.seed(129)
m = 10
n = 5
X = np.random.rand(m, n)
Y = np.random.rand(m, n) * 0.1
R = align_embeddings(X, Y)

Loss at iteration 0 i 3.7241601261682264
Loss at iteration 25 i 3.628289198210137
Loss at iteration 50 i 3.53497687566553
Loss at iteration 75 i 3.444154562048557


In [75]:
w4_unittest.test_align_embeddings(align_embeddings)

 All tests passed


In [76]:
R_train = align_embeddings(X_train, Y_train, train_steps=400, learning_rate=0.8)

Loss at iteration 0 i 963.0145619955867
Loss at iteration 25 i 97.82916853030906
Loss at iteration 50 i 26.832879686583038
Loss at iteration 75 i 9.789329539535158
Loss at iteration 100 i 4.377647478540068
Loss at iteration 125 i 2.3280536915224053
Loss at iteration 150 i 1.4479896206566953
Loss at iteration 175 i 1.0337809042840693
Loss at iteration 200 i 0.8251343300366362
Loss at iteration 225 i 0.7144881382598546
Loss at iteration 250 i 0.6533957223255501
Loss at iteration 275 i 0.6185348691701879
Loss at iteration 300 i 0.5980814008397813
Loss at iteration 325 i 0.5857882199933446
Loss at iteration 350 i 0.578241077375853
Loss at iteration 375 i 0.5735195038803573


## Testing the translation with K-nearest neighbours algorithm

In [77]:
def nearest_neighbor(v, candidates, k=1, cosine_similarity=cosine_similarity):
    """
    Input:
      - v, the vector you are going find the nearest neighbor for
      - candidates: a set of vectors where we will find the neighbors
      - k: top k nearest neighbors to find
    Output:
      - k_idx: the indices of the top k closest vectors in sorted form
    """
    ### START CODE HERE ###
    similarity_l = []

    for candidate in candidates:
        similarity = cosine_similarity(candidate, v)
        similarity_l.append(similarity)

    sorted_ids = np.argsort(similarity_l)

    k_idx = sorted_ids[::-1][:k]

    return k_idx

In [100]:
# Test your implementation:
v = np.array([1, 0, 1])
candidates = np.array([[1, 0, 5], [-2, 5, 3], [2, 0, 1], [6, -9, 5], [9, 9, 9]])
print(candidates)
# print(candidates[nearest_neighbor(v, candidates, 3)])

[[ 1  0  5]
 [-2  5  3]
 [ 2  0  1]
 [ 6 -9  5]
 [ 9  9  9]]


In [79]:
w4_unittest.test_nearest_neighbor(nearest_neighbor)

 All tests passed


## Test your translation and compute its accuracy

In [97]:
def test_vocabulary(X, Y, R, nearest_neighbor=nearest_neighbor):
    """
    Input:
        X: a matrix where the columns are the English embeddings.
        Y: a matrix where the columns correspong to the French embeddings.
        R: the transform matrix which translates word embeddings from
        English to French word vector space.
    Output:
        accuracy: for the English to French capitals
    """

    ### START CODE HERE ###
    # The prediction is X times R
    print(f"Shape X: {X.shape}")
    print(f"Shape Y: {Y.shape}")
    print(f"Shape R: {R.shape}")
    pred = X @ R
    print(f"Shape pred: {pred.shape}")

    # initialize the number correct to zero
    num_correct = 0

    print(f"Len pred: {len(pred)}")

    for i in range(len(pred)):
        pred_idx = nearest_neighbor(v=pred[i, :], candidates=Y)

        if pred_idx == i:
            num_correct += 1

    accuracy = num_correct / pred.shape[0]

    return accuracy

In [98]:
X_val, Y_val = get_matrices(en_fr_test, fr_embeddings_subset, en_embeddings_subset)

(1438, 300)
(1438, 300)


In [99]:
acc = test_vocabulary(X_val, Y_val, R_train)  # this might take a minute or two
print(f"accuracy on test set is {acc:.3f}")

Shape X: (1438, 300)
Shape Y: (1438, 300)
Shape R: (300, 300)
Shape pred: (1438, 300)
Len pred: 1438
accuracy on test set is 0.557


In [ ]:
def align_embeddings(
    X,
    Y,
    train_steps=100,
    learning_rate=0.0003,
    verbose=True,
    compute_loss=compute_loss,
    compute_gradient=compute_gradient,
):
    """
    Inputs:
        X: a matrix of dimension (m,n) where the columns are the English embeddings. ---> Rows m words, Cols n embeedding features
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        train_steps: positive int - describes how many steps will gradient descent algorithm do.
        learning_rate: positive float - describes how big steps will  gradient descent algorithm do.
    Outputs:
        R: a matrix of dimension (n,n) - the projection matrix that minimizes the F norm ||X R -Y||^2
    """
    np.random.seed(129)

    R = np.random.rand(X.shape[1], X.shape[1])

    for i in range(train_steps):
        if verbose and i % 25 == 0:
            print(f"Loss at iteration {i} i {compute_loss(X, Y, R)}")

        gradient = compute_gradient(X, Y, R)
        R = R - learning_rate * gradient

    return R

In [ ]:
np.random.seed(129)
m = 10
n = 5
X = np.random.rand(m, n)
Y = np.random.rand(m, n) * 0.1
R = align_embeddings(X, Y)

Loss at iteration 0 i 3.7241601261682264
Loss at iteration 25 i 3.628289198210137
Loss at iteration 50 i 3.53497687566553
Loss at iteration 75 i 3.444154562048557


In [ ]:
w4_unittest.test_align_embeddings(align_embeddings)

 All tests passed


In [ ]:
R_train = align_embeddings(X_train, Y_train, train_steps=400, learning_rate=0.8)

Loss at iteration 0 i 963.0145619955867
Loss at iteration 25 i 97.82916853030906
Loss at iteration 50 i 26.832879686583038
Loss at iteration 75 i 9.789329539535158
Loss at iteration 100 i 4.377647478540068
Loss at iteration 125 i 2.3280536915224053
Loss at iteration 150 i 1.4479896206566953
Loss at iteration 175 i 1.0337809042840693
Loss at iteration 200 i 0.8251343300366362
Loss at iteration 225 i 0.7144881382598546
Loss at iteration 250 i 0.6533957223255501
Loss at iteration 275 i 0.6185348691701879
Loss at iteration 300 i 0.5980814008397813
Loss at iteration 325 i 0.5857882199933446
Loss at iteration 350 i 0.578241077375853
Loss at iteration 375 i 0.5735195038803573


## Testing the translation with K-nearest neighbours algorithm

In [ ]:
def nearest_neighbor(v, candidates, k=1, cosine_similarity=cosine_similarity):
    """
    Input:
      - v, the vector you are going find the nearest neighbor for
      - candidates: a set of vectors where we will find the neighbors
      - k: top k nearest neighbors to find
    Output:
      - k_idx: the indices of the top k closest vectors in sorted form
    """
    ### START CODE HERE ###
    similarity_l = []

    for candidate in candidates:
        similarity = cosine_similarity(candidate, v)
        similarity_l.append(similarity)

    sorted_ids = np.argsort(similarity_l)

    k_idx = sorted_ids[::-1][:k]

    return k_idx

In [ ]:
# Test your implementation:
v = np.array([1, 0, 1])
candidates = np.array([[1, 0, 5], [-2, 5, 3], [2, 0, 1], [6, -9, 5], [9, 9, 9]])
print(candidates)
# print(candidates[nearest_neighbor(v, candidates, 3)])

[[ 1  0  5]
 [-2  5  3]
 [ 2  0  1]
 [ 6 -9  5]
 [ 9  9  9]]


In [ ]:
w4_unittest.test_nearest_neighbor(nearest_neighbor)

 All tests passed


## Test your translation and compute its accuracy

In [ ]:
def test_vocabulary(X, Y, R, nearest_neighbor=nearest_neighbor):
    """
    Input:
        X: a matrix where the columns are the English embeddings.
        Y: a matrix where the columns correspong to the French embeddings.
        R: the transform matrix which translates word embeddings from
        English to French word vector space.
    Output:
        accuracy: for the English to French capitals
    """

    ### START CODE HERE ###
    # The prediction is X times R
    print(f"Shape X: {X.shape}")
    print(f"Shape Y: {Y.shape}")
    print(f"Shape R: {R.shape}")
    pred = X @ R
    print(f"Shape pred: {pred.shape}")

    # initialize the number correct to zero
    num_correct = 0

    print(f"Len pred: {len(pred)}")

    for i in range(len(pred)):
        pred_idx = nearest_neighbor(v=pred[i, :], candidates=Y)

        if pred_idx == i:
            num_correct += 1

    accuracy = num_correct / pred.shape[0]

    return accuracy

In [ ]:
X_val, Y_val = get_matrices(en_fr_test, fr_embeddings_subset, en_embeddings_subset)

(1438, 300)
(1438, 300)


In [ ]:
acc = test_vocabulary(X_val, Y_val, R_train)  # this might take a minute or two
print(f"accuracy on test set is {acc:.3f}")

Shape X: (1438, 300)
Shape Y: (1438, 300)
Shape R: (300, 300)
Shape pred: (1438, 300)
Len pred: 1438
accuracy on test set is 0.557


In [101]:
w4_unittest.unittest_test_vocabulary(test_vocabulary)

Shape X: (2, 1)
Shape Y: (2, 1)
Shape R: (1, 1)
Shape pred: (2, 1)
Len pred: 2
Shape X: (4, 2)
Shape Y: (4, 2)
Shape R: (2, 2)
Shape pred: (4, 2)
Len pred: 4
 All tests passed


## LSH and Document Search

In [103]:
all_positive_tweets = twitter_samples.strings("positive_tweets.json")
all_negative_tweets = twitter_samples.strings("negative_tweets.json")
all_tweets = all_positive_tweets + all_negative_tweets

In [104]:
def get_document_embedding(tweet, en_embeddings, process_tweet=process_tweet):
    """
    Input:
        - tweet: a string
        - en_embeddings: a dictionary of word embeddings
    Output:
        - doc_embedding: sum of all word embeddings in the tweet
    """

    doc_embedding = np.zeros(300)
    processed_doc = process_tweet(tweet)

    for word in processed_doc:
        doc_embedding += en_embeddings.get(word, 0)

    return doc_embedding

In [105]:
custom_tweet = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"
tweet_embedding = get_document_embedding(custom_tweet, en_embeddings_subset)
tweet_embedding[-5:]

array([-0.00268555, -0.15378189, -0.55761719, -0.07216644, -0.32263184])

In [106]:
w4_unittest.test_get_document_embedding(get_document_embedding)

 All tests passed


In [108]:
def get_document_vecs(
    all_docs, en_embeddings, get_document_embedding=get_document_embedding
):
    """
    Input:
        - all_docs: list of strings - all tweets in our dataset.
        - en_embeddings: dictionary with words as the keys and their embeddings as the values.
    Output:
        - document_vec_matrix: matrix of tweet embeddings.
        - ind2Doc_dict: dictionary with indices of tweets in vecs as keys and their embeddings as the values.
    """
    ind2Doc_dict = {}
    document_vec_l = []

    for i, doc in enumerate(all_docs):
        document_embedding = get_document_embedding(doc, en_embeddings)
        document_vec_l.append(document_embedding)
        ind2Doc_dict[i] = document_embedding

    document_vec_matrix = np.array(document_vec_l)

    return document_vec_matrix, ind2Doc_dict

In [109]:
document_vecs, ind2Tweet = get_document_vecs(all_tweets, en_embeddings_subset)

In [110]:
print(f"length of dictionary {len(ind2Tweet)}")
print(f"shape of document_vecs {document_vecs.shape}")

length of dictionary 10000
shape of document_vecs (10000, 300)


In [111]:
w4_unittest.test_get_document_vecs(get_document_vecs)

 All tests passed


In [112]:
my_tweet = "i am sad"
process_tweet(my_tweet)
tweet_embedding = get_document_embedding(my_tweet, en_embeddings_subset)

In [114]:
similarity = cosine_similarity(document_vecs, tweet_embedding)
similarity

array([0.12436915, 0.34844389, 0.23104973, ..., 0.        , 0.13630579,
       0.09393305])

In [117]:
idx = np.argmax(similarity)
idx

5202

In [118]:
all_tweets[idx]

'@hanbined sad pray for me :((('

### Finding the most similar tweet with LSH

You will now implement locality sensitive hashing (LSH) to identify the most similar tweet.
* Instead of looking at all 10,000 vectors, you can just search a subset to find
its nearest neighbors.


In [119]:
N_VECS = len(all_tweets)  # This many vectors.
N_DIMS = len(ind2Tweet[1])  # Vector dimensionality.
print(f"Number of vectors is {N_VECS} and each has {N_DIMS} dimensions.")

Number of vectors is 10000 and each has 300 dimensions.


#### Choosing the number of planes

* Each plane divides the space to $2$ parts.
* So $n$ planes divide the space into $2^{n}$ hash buckets.
* We want to organize 10,000 document vectors into buckets so that every bucket has about $~16$ vectors.
* For that we need $\frac{10000}{16}=625$ buckets.
* We're interested in $n$, number of planes, so that $2^{n}= 625$. Now, we can calculate $n=\log_{2}625 = 9.29 \approx 10$.

In [120]:
# The number of planes. We use log2(625) to have ~16 vectors/bucket.
N_PLANES = 10
# Number of times to repeat the hashing to improve the search.
N_UNIVERSES = 25

In [121]:
np.random.seed(0)
planes_l = [np.random.normal(size=(N_DIMS, N_PLANES)) for _ in range(N_UNIVERSES)]

In [122]:
def hash_value_of_vector(v, planes):
    """Create a hash for a vector; hash_id says which random hash to use.
    Input:
        - v:  vector of tweet. It's dimension is (1, N_DIMS)
        - planes: matrix of dimension (N_DIMS, N_PLANES) - the set of planes that divide up the region
    Output:
        - res: a number which is used as a hash for your vector

    """
    ### START CODE HERE ###
    # for the set of planes,
    # calculate the dot product between the vector and the matrix containing the planes
    # remember that planes has shape (300, 10)
    # The dot product will have the shape (1,10)
    # planes: [N_DIMS, N_PLANES]
    # v: [1 , N_DIMS]
    # planes x v.T -> [N_DIMS, N_PLANES] x [N_DIMS, 1]
    dot_product = np.dot(v, planes)

    # get the sign of the dot product (1,10) shaped vector
    sign_of_dot_product = np.sign(dot_product)

    # set h to be false (eqivalent to 0 when used in operations) if the sign is negative,
    # and true (equivalent to 1) if the sign is positive (1,10) shaped vector
    # if the sign is 0, i.e. the vector is in the plane, consider the sign to be positive
    h = np.where(sign_of_dot_product >= 0, 1, 0)

    # remove extra un-used dimensions (convert this from a 2D to a 1D array)
    h = np.squeeze(h)

    # initialize the hash value to 0
    hash_value = 0

    n_planes = planes.shape[1]  # N_PLANES
    for i in range(n_planes):
        # increment the hash value by 2^i * h_i
        hash_value += 2**i * h[i]

    ### END CODE HERE ###

    # cast hash_value as an integer
    hash_value = int(hash_value)

    return hash_value

In [123]:
np.random.seed(0)
idx = 0
planes = planes_l[idx]  # get one 'universe' of planes to test the function
vec = np.random.rand(1, 300)
print(
    f" The hash value for this vector,",
    f"and the set of planes at index {idx},",
    f"is {hash_value_of_vector(vec, planes)}",
)

 The hash value for this vector, and the set of planes at index 0, is 768


In [124]:
w4_unittest.test_hash_value_of_vector(hash_value_of_vector)

 All tests passed


In [125]:
def make_hash_table(vecs, planes, hash_value_of_vector=hash_value_of_vector):
    """
    Input:
        - vecs: list of vectors to be hashed.
        - planes: the matrix of planes in a single "universe", with shape (embedding dimensions, number of planes).
    Output:
        - hash_table: dictionary - keys are hashes, values are lists of vectors (hash buckets)
        - id_table: dictionary - keys are hashes, values are list of vectors id's
                            (it's used to know which tweet corresponds to the hashed vector)
    """
    # number of planes is the number of columns in the planes matrix
    num_of_planes = planes.shape[1]

    # number of buckets is 2^(number of planes)
    # ALTERNATIVE SOLUTION COMMENT:
    # num_buckets = pow(2, num_of_planes)
    num_buckets = 2**num_of_planes

    # create the hash table as a dictionary.
    # Keys are integers (0,1,2.. number of buckets)
    # Values are empty lists
    hash_table = {i: [] for i in range(num_buckets)}

    # create the id table as a dictionary.
    # Keys are integers (0,1,2... number of buckets)
    # Values are empty lists
    id_table = {i: [] for i in range(num_buckets)}

    # for each vector in 'vecs'
    for i, v in enumerate(vecs):
        # calculate the hash value for the vector
        h = hash_value_of_vector(v, planes)

        # store the vector into hash_table at key h,
        # by appending the vector v to the list at key h
        hash_table[h].append(v)  # @REPLACE None

        # store the vector's index 'i' (each document is given a unique integer 0,1,2...)
        # the key is the h, and the 'i' is appended to the list at key h
        id_table[h].append(i)  # @REPLACE None

    return hash_table, id_table

In [126]:
planes = planes_l[0]  # get one 'universe' of planes to test the function
tmp_hash_table, tmp_id_table = make_hash_table(document_vecs, planes)

print(f"The hash table at key 0 has {len(tmp_hash_table[0])} document vectors")
print(f"The id table at key 0 has {len(tmp_id_table[0])} document indices")
print(
    f"The first 5 document indices stored at key 0 of id table are {tmp_id_table[0][0:5]}"
)

The hash table at key 0 has 3 document vectors
The id table at key 0 has 3 document indices
The first 5 document indices stored at key 0 of id table are [3276, 3281, 3282]


In [127]:
def create_hash_id_tables(n_universes):
    hash_tables = []
    id_tables = []
    for universe_id in range(n_universes):  # there are 25 hashes
        print("working on hash universe #:", universe_id)
        planes = planes_l[universe_id]
        hash_table, id_table = make_hash_table(document_vecs, planes)
        hash_tables.append(hash_table)
        id_tables.append(id_table)

    return hash_tables, id_tables


hash_tables, id_tables = create_hash_id_tables(N_UNIVERSES)

working on hash universe #: 0
working on hash universe #: 1
working on hash universe #: 2
working on hash universe #: 3
working on hash universe #: 4
working on hash universe #: 5
working on hash universe #: 6
working on hash universe #: 7
working on hash universe #: 8
working on hash universe #: 9
working on hash universe #: 10
working on hash universe #: 11
working on hash universe #: 12
working on hash universe #: 13
working on hash universe #: 14
working on hash universe #: 15
working on hash universe #: 16
working on hash universe #: 17
working on hash universe #: 18
working on hash universe #: 19
working on hash universe #: 20
working on hash universe #: 21
working on hash universe #: 22
working on hash universe #: 23
working on hash universe #: 24


In [128]:
def approximate_knn(
    doc_id,
    v,
    planes_l,
    hash_tables,
    id_tables,
    k=1,
    num_universes_to_use=25,
    hash_value_of_vector=hash_value_of_vector,
):
    """Search for k-NN using hashes."""
    # assert num_universes_to_use <= N_UNIVERSES

    # Vectors that will be checked as possible nearest neighbor
    vecs_to_consider_l = list()

    # list of document IDs
    ids_to_consider_l = list()

    # create a set for ids to consider, for faster checking if a document ID already exists in the set
    ids_to_consider_set = set()

    # loop through the universes of planes
    for universe_id in range(num_universes_to_use):
        # get the set of planes from the planes_l list, for this particular universe_id
        planes = planes_l[universe_id]

        # get the hash value of the vector for this set of planes
        hash_value = hash_value_of_vector(v, planes)

        # get the hash table for this particular universe_id
        hash_table = hash_tables[universe_id]

        # get the list of document vectors for this hash table, where the key is the hash_value
        document_vectors_l = hash_table[hash_value]

        # get the id_table for this particular universe_id
        id_table = id_tables[universe_id]

        # get the subset of documents to consider as nearest neighbors from this id_table dictionary
        new_ids_to_consider = id_table[hash_value]

        ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

        # loop through the subset of document vectors to consider
        for i, new_id in enumerate(new_ids_to_consider):
            if doc_id == new_id:
                continue

            # if the document ID is not yet in the set ids_to_consider...
            if new_id not in ids_to_consider_set:
                # access document_vectors_l list at index i to get the embedding
                # then append it to the list of vectors to consider as possible nearest neighbors
                document_vector_at_i = document_vectors_l[i]
                vecs_to_consider_l.append(document_vector_at_i)

                # append the new_id (the index for the document) to the list of ids to consider
                ids_to_consider_l.append(new_id)

                # also add the new_id to the set of ids to consider
                # (use this to check if new_id is not already in the IDs to consider)
                ids_to_consider_set.add(new_id)

        ### END CODE HERE ###

    # Now run k-NN on the smaller set of vecs-to-consider.
    print("Fast considering %d vecs" % len(vecs_to_consider_l))

    # convert the vecs to consider set to a list, then to a numpy array
    vecs_to_consider_arr = np.array(vecs_to_consider_l)

    # call nearest neighbors on the reduced list of candidate vectors
    nearest_neighbor_idx_l = nearest_neighbor(v, vecs_to_consider_arr, k=k)

    # Use the nearest neighbor index list as indices into the ids to consider
    # create a list of nearest neighbors by the document ids
    nearest_neighbor_ids = [ids_to_consider_l[idx] for idx in nearest_neighbor_idx_l]

    return nearest_neighbor_ids

In [129]:
doc_id = 0
doc_to_search = all_tweets[doc_id]
vec_to_search = document_vecs[doc_id]

In [132]:
nearest_neighbor_ids = approximate_knn(
    doc_id, vec_to_search, planes_l, hash_tables, id_tables, k=3, num_universes_to_use=5
)

Fast considering 77 vecs


In [133]:
print(f"Nearest neighbors for document {doc_id}")
print(f"Document contents: {doc_to_search}")
print("")

for neighbor_id in nearest_neighbor_ids:
    print(f"Nearest neighbor at document id {neighbor_id}")
    print(f"document contents: {all_tweets[neighbor_id]}")

Nearest neighbors for document 0
Document contents: #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

Nearest neighbor at document id 51
document contents: #FollowFriday @France_Espana @reglisse_menthe @CCI_inter for being top engaged members in my community this week :)
Nearest neighbor at document id 2478
document contents: #ShareTheLove @oymgroup @musicartisthere for being top HighValue members this week :) @nataliavas http://t.co/IWSDMtcayt
Nearest neighbor at document id 105
document contents: #FollowFriday @straz_das @DCarsonCPA @GH813600 for being top engaged members in my community this week :)
